# RESULTS EXPLORATION

In [2]:
# definisco una funzione che data path detecta l'autore
import os
def author_detect(path:str):
    # l'autore è il nome della cartella contenente il file
    author = os.path.basename(os.path.dirname(path))
    return author

# carico il file csv distances.csv
import pandas as pd
import numpy
from matplotlib import pyplot as plt

# il file csv è una matrice di float con righe e colonne indicizzate con una path
df = pd.read_csv('distances.csv', index_col=0)

distances = df.to_numpy()

# print no-nan values quantity
print((numpy.count_nonzero(~numpy.isnan(distances)) - 420)//2)

# creo una tabella dove inserisco i risultati:
# la riga dell'autore i-esimo e la colonna dell'autore j-esimo contiene il numero di opere di i che sono state classificate come opere di j

# creo una lista di autori
authors = df.index.map(author_detect).unique()

# creo una tabella vuota
table = pd.DataFrame(numpy.zeros((len(authors), len(authors))), index=authors, columns=authors)

deep = 1
for i in range(len(df)):
    # estraggo le distanze della i-esima opera
    work_distances = distances[i].copy()
    # trasformo i nan in inf
    work_distances = numpy.nan_to_num(work_distances, nan=float('inf'))
    # trasformo il confronto con se stesso in inf
    work_distances[i] = float('inf')
    # trovo l'indice delle deep opere più vicine
    nearest_work = numpy.argpartition(work_distances, deep)[:deep]
    
    # per ogni autore conto il numero di opere in questa lista
    # vettore di infiniti
    score = numpy.full(len(authors), numpy.inf)
    counter = numpy.zeros(len(authors))
    for j in range(deep):
        # ricavo l'autore dell'opera di indice nearest_work[j]
        nearest_author = author_detect(df.index[nearest_work[j]])
        # incremento lo score con la distanza
        # se ho infinito annullo
        if score[authors.get_loc(nearest_author)] == numpy.inf:
            score[authors.get_loc(nearest_author)] = 0
        score[authors.get_loc(nearest_author)] += distances[i, nearest_work[j]]
        # incremento il contatore
        counter[authors.get_loc(nearest_author)] += 1
    
    # ricavo l'autore dell'opera stessa
    author = author_detect(df.index[i])
    # trovo l'autore dell'opera più vicina
    score = score / counter
    nearest_author = authors[score.argmin()]
    # incremento il contatore dell'autore i-esimo e dell'autore più vicino con il rapporto del numero di opere considerate
    # per l'opera i-esima conto il numero di opere che sono state comparate (non sono inf o nan)
    num_of_works = numpy.count_nonzero(work_distances != numpy.inf)
    table.at[author, nearest_author] += 1 if num_of_works > 62 else 0

print(table)

# per ogni autore calcolo i veri positivi e i veri negativi
for i in range(len(authors)):
    fp = 1 - table.iat[i, i] / table.iloc[:, i].sum()
    fn = 1 - table.iat[i, i] / table.iloc[i].sum()
    print('')
    print(f'Author {authors[i]}')
    print(f'False Positives: {fp}')
    print(f'False Negatives: {fn}')
    p = (table.iloc[i, :] / table.iloc[i, :].sum()).values
    p = p[p != 0]
    entropy = -numpy.log(p) @ p
    print(f'Entropy: {entropy} (max: {numpy.log(len(authors))*len(authors)})')

13169
         Author2  Author4  Author1  Author3
Author2      5.0      0.0      0.0      0.0
Author4      0.0      4.0      8.0      0.0
Author1      2.0      1.0     36.0      2.0
Author3      0.0      0.0      3.0      2.0

Author Author2
False Positives: 0.2857142857142857
False Negatives: 0.0
Entropy: 0.0 (max: 5.545177444479562)

Author Author4
False Positives: 0.19999999999999996
False Negatives: 0.6666666666666667
Entropy: 0.6365141682948129 (max: 5.545177444479562)

Author Author1
False Positives: 0.23404255319148937
False Negatives: 0.12195121951219512
Entropy: 0.4994435177613873 (max: 5.545177444479562)

Author Author3
False Positives: 0.5
False Negatives: 0.6
Entropy: 0.6730116670092564 (max: 5.545177444479562)


# SVD

In [ ]:
# carico l'immagine fraseGauss.png
from PIL import Image
import matplotlib.pyplot as plt

# converto in bit 0, 1
image = Image.open('fraseGauss_rotated.png').convert('L')
# aumento il contrasto
#image = image.point(lambda p: (p / 255)**5 * 255)
#image.save('fraseGauss.png')

plt.figure()
plt.imshow(image, cmap='gray', vmin=0, vmax=256)
plt.axis('off')
plt.show()

# uso SVD per comprimere l'immagine
import numpy as np

image_np = np.array(image)
U, S, V = np.linalg.svd(image_np, full_matrices=False)

# comprimo l'immagine
k=20
compressed_image = U[:, k:] @ np.diag(S[k:]) @ V[k:, :]
plt.figure()
plt.imshow(compressed_image, cmap='gray')
plt.axis('off')
plt.imsave('first_pattern.png', compressed_image, dpi=600, cmap='gray')
plt.show()
